In [1]:
from torch import cuda
cuda.empty_cache()

## Imports

In [2]:
import os
import numpy as np
import random
import h5py
import torch
import cv2
import glob
import torch.utils.data as udata
import torch
import torch.nn as nn
import cv2
import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

## Utils

In [3]:
def normalize(data):
    return data/255.0 


def Im2Patch(img, win, stride=1):
    k = 0
    endc = img.shape[0]
    endw = img.shape[1]
    endh = img.shape[2]
    patch = img[:, 0 : endw - win + 0 + 1 : stride, 0 : endh - win + 0 + 1 : stride]
    TotalPatNum = patch.shape[1] * patch.shape[2]
    Y = np.zeros([endc, win * win, TotalPatNum], np.float32)
    for i in range(win):
        for j in range(win):
            patch = img[
                :, i : endw - win + i + 1 : stride, j : endh - win + j + 1 : stride
            ]
            Y[:, k, :] = np.array(patch[:]).reshape(endc, TotalPatNum)
            k += 1
    return Y.reshape([endc, win, win, TotalPatNum])

In [4]:
def weights_init_kaiming(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)


def batch_PSNR(img1, img2, scale):
    img1 = img1.cpu().detach().numpy()
    img2 = img2.cpu().detach().numpy()
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 1.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

In [5]:
def data_augmentation(data, aug_type):
    if aug_type == 1:
        return np.fliplr(data)
    elif aug_type == 2:
        return np.flipud(data)
    elif aug_type == 3:
        return np.rot90(data, k=1)
    elif aug_type == 4:
        return np.rot90(data, k=2)
    elif aug_type == 5:
        return np.rot90(data, k=3)
    elif aug_type == 6:
        return np.fliplr(np.rot90(data, k=1))
    elif aug_type == 7:
        return np.flipud(np.rot90(data, k=1))
    elif aug_type == 8:
        return np.fliplr(np.flipud(data))
    else:
        return data

In [6]:
def prepare_data(data_path, patch_size, stride, aug_times=1):
    print("Process training data")
    scales = [1, 0.9, 0.8, 0.7]
    files = glob.glob(os.path.join(data_path, "train", "*.png"))
    files.sort()
    h5f = h5py.File("train.h5", "w")
    train_num = 0
    for i in range(len(files)):
        img = cv2.imread(files[i])
        h, w, c = img.shape
        for k in range(len(scales)):
            Img = cv2.resize(
                img,
                (int(h * scales[k]), int(w * scales[k])),
                interpolation=cv2.INTER_CUBIC,
            )
            Img = np.expand_dims(Img[:, :, 0].copy(), 0)
            Img = np.float32(normalize(Img))
            patches = Im2Patch(Img, win=patch_size, stride=stride)
            print(
                f"File: {files[i]} Scale {scales[k]:.1f} # samples: {patches.shape[3] * aug_times}"
            )
            for n in range(patches.shape[3]):
                data = patches[:, :, :, n].copy()
                h5f.create_dataset(str(train_num), data=data)
                train_num += 1
                for m in range(aug_times - 1):
                    data_aug = data_augmentation(data, np.random.randint(1, 8))
                    h5f.create_dataset(str(train_num) + f"_aug_{m+1}", data=data_aug)
                    train_num += 1
    h5f.close()

    # Validation data
    print("\nProcess validation data")
    files = glob.glob(os.path.join(data_path, "Set12", "*.png"))
    files.sort()
    h5f = h5py.File("val.h5", "w")
    val_num = 0
    for i in range(len(files)):
        print(f"File: {files[i]}")
        img = cv2.imread(files[i])
        img = np.expand_dims(img[:, :, 0], 0)
        img = np.float32(normalize(img))
        h5f.create_dataset(str(val_num), data=img)
        val_num += 1
    h5f.close()
    print(f"Training set, # samples {train_num}\n")
    print(f"Validation set, # samples {val_num}\n")

In [7]:
class Dataset(udata.Dataset):
    def __init__(self, train=True):
        super(Dataset, self).__init__()
        self.train = train
        if self.train:
            self.h5f = h5py.File("train.h5", "r")
        else:
            self.h5f = h5py.File("val.h5", "r")
        self.keys = list(self.h5f.keys())
        random.shuffle(self.keys)
        self.h5f.close()

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, index):
        if self.train:
            h5f = h5py.File("train.h5", "r")
        else:
            h5f = h5py.File("val.h5", "r")
        key = self.keys[index]
        data = np.array(h5f[key])
        h5f.close()
        return torch.Tensor(data)

In [8]:


class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = [
            nn.Conv2d(
                in_channels=channels,
                out_channels=features,
                kernel_size=kernel_size,
                padding=padding,
                bias=False,
            ),
            nn.ReLU(inplace=True),
        ]
        for _ in range(num_of_layers - 2):
            layers.extend(
                [
                    nn.Conv2d(
                        in_channels=features,
                        out_channels=features,
                        kernel_size=kernel_size,
                        padding=padding,
                        bias=False,
                    ),
                    nn.BatchNorm2d(features),
                    nn.ReLU(inplace=True),
                ]
            )
        layers.append(
            nn.Conv2d(
                in_channels=features,
                out_channels=channels,
                kernel_size=kernel_size,
                padding=padding,
                bias=False,
            )
        )
        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        return self.dncnn(x)

In [9]:
from torch.utils.data import Dataset
from PIL import Image
import os
import torch
from torchvision import transforms


class DenoisingDataset(Dataset):
    def __init__(self, noisy_dir, clean_dir, transform=None):
        self.noisy_dir = noisy_dir
        self.clean_dir = clean_dir
        self.transform = transform

        # Get list of noisy and clean image files
        self.noisy_images = sorted(os.listdir(noisy_dir))
        self.clean_images = sorted(os.listdir(clean_dir))

        # Ensure same number of noisy and clean images
        if len(self.noisy_images) != len(self.clean_images):
            raise ValueError("The number of noisy and clean images must be the same.")

    def __len__(self):
        return len(self.noisy_images)

    def __getitem__(self, idx):
        noisy_image_path = os.path.join(self.noisy_dir, self.noisy_images[idx])
        clean_image_path = os.path.join(self.clean_dir, self.clean_images[idx])

        noisy_image = Image.open(noisy_image_path)
        clean_image = Image.open(clean_image_path)

        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform(clean_image)

        return noisy_image, clean_image

In [10]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from torchvision import  transforms
from torch.utils.data import DataLoader
from torchvision import transforms


def train_dncnn(
    preprocess=False,
    batch_size=128,
    num_of_layers=17,
    epochs=50,
    lr=1e-3,
    milestone=30,
    noiseL=25,
    val_noiseL=25,
    logdir="logs",
):
    if preprocess:
        prepare_data("data", patch_size=40, stride=10, aug_times=1)

    print("Loading dataset ...")

    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),  # Adjust to your model's input size
            transforms.ToTensor(),
            transforms.Grayscale(num_output_channels=1),
        ]
    )

    # Directory paths
    train_noisy_dir = (
        "/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/train/noisy"
    )
    train_clean_dir = (
        "/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/train/clean"
    )
    valid_noisy_dir = "/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/valid/noisy"  # Add validation directories if you have them
    valid_clean_dir = (
        "/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/valid/clean"
    )

    # Load datasets
    dataset_train = DenoisingDataset(
        noisy_dir=train_noisy_dir, clean_dir=train_clean_dir, transform=transform
    )
    dataset_val = DenoisingDataset(
        noisy_dir=valid_noisy_dir, clean_dir=valid_clean_dir, transform=transform
    )
    loader_train = DataLoader(
        dataset=dataset_train, batch_size=batch_size, shuffle=True, num_workers=4
    )
    loader_val = DataLoader(
        dataset=dataset_val, batch_size=batch_size, shuffle=False, num_workers=4
    )

    print(f"# of training samples: {len(dataset_train)}")
    print(f"# of validation samples: {len(dataset_val)}")

    net = DnCNN(channels=1, num_of_layers=num_of_layers)
    net.apply(weights_init_kaiming)
    criterion = nn.MSELoss(reduction="sum")

    model = nn.DataParallel(net).cuda()
    criterion.cuda()

    optimizer = optim.Adam(model.parameters(), lr=lr)
    writer = SummaryWriter(logdir)

    for epoch in range(epochs):
        current_lr = lr if epoch < milestone else lr / 10.0
        for param_group in optimizer.param_groups:
            param_group["lr"] = current_lr

        print(f"Epoch {epoch+1}, Learning Rate: {current_lr}")

        # Training phase
        net.train()
        running_loss = 0.0
        for i, (noisy_img, clean_img) in enumerate(loader_train):
            noisy_img, clean_img = noisy_img.cuda(), clean_img.cuda()

            optimizer.zero_grad()

            # Forward pass
            output = net(noisy_img)
            loss = criterion(output, clean_img)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (i + 1) % 100 == 0:  # Print every 100 batches
                print(f"Batch {i+1}, Loss: {running_loss / (i + 1)}")

        # Log training loss
        writer.add_scalar("Loss/train", running_loss / len(loader_train), epoch)

        # Validation phase
        net.eval()
        with torch.no_grad():
            psnr = 0.0
            for i, (noisy_img, clean_img) in enumerate(loader_val):
                noisy_img, clean_img = noisy_img.cuda(), clean_img.cuda()

                output = net(noisy_img)
                psnr += batch_PSNR(output, clean_img, scale=1)

            avg_psnr = psnr / len(dataset_val)
            print(f"Validation PSNR: {avg_psnr}")

            # Log validation PSNR
            writer.add_scalar("PSNR/val", avg_psnr, epoch)

        # Save the model checkpoint
        if (epoch + 1) % 10 == 0:  # Save every 10 epochs
            checkpoint_path = f"{logdir}/model_epoch_{epoch + 1}.pth"
            torch.save(net.state_dict(), checkpoint_path)
            print(f"Saved checkpoint: {checkpoint_path}")

    writer.close()
    print("Training finished.")

In [11]:
train_dncnn(
    preprocess=True,
    batch_size=2,
    num_of_layers=17,
    epochs=10,
    lr=1e-3,
    milestone=30,
    noiseL=25,
    val_noiseL=25,
    logdir="logs",
)

Process training data
File: data/train/test_001.png Scale 1.0 # samples: 225
File: data/train/test_001.png Scale 0.9 # samples: 169
File: data/train/test_001.png Scale 0.8 # samples: 121
File: data/train/test_001.png Scale 0.7 # samples: 81
File: data/train/test_002.png Scale 1.0 # samples: 225
File: data/train/test_002.png Scale 0.9 # samples: 169
File: data/train/test_002.png Scale 0.8 # samples: 121
File: data/train/test_002.png Scale 0.7 # samples: 81
File: data/train/test_003.png Scale 1.0 # samples: 225
File: data/train/test_003.png Scale 0.9 # samples: 169
File: data/train/test_003.png Scale 0.8 # samples: 121
File: data/train/test_003.png Scale 0.7 # samples: 81
File: data/train/test_004.png Scale 1.0 # samples: 225
File: data/train/test_004.png Scale 0.9 # samples: 169
File: data/train/test_004.png Scale 0.8 # samples: 121
File: data/train/test_004.png Scale 0.7 # samples: 81
File: data/train/test_005.png Scale 1.0 # samples: 225
File: data/train/test_005.png Scale 0.9 # sampl

OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 5.62 GiB total capacity; 149.92 MiB already allocated; 36.50 MiB free; 160.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def normalize(data):
    return data / 255.0


import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F


# Define or import your DnCNN model here
class DnCNN(nn.Module):
    # Define the model architecture here
    pass


def normalize(data):
    return data / 255.0


def batch_PSNR(output, target, max_val):
    mse = F.mse_loss(output, target, reduction="mean")
    psnr = 10 * torch.log10((max_val**2) / mse)
    return psnr.item()


def test_dncnn(test_data="Set12", num_of_layers=17, logdir="logs"):
    print("Loading model ...")
    net = DnCNN(channels=1, num_of_layers=num_of_layers)
    model = net.cuda()
    model.load_state_dict(torch.load(os.path.join(logdir, "model_epoch_10.pth")))
    model.eval()

    # Paths for clean and noisy images
    clean_folder = (
        "/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/clean"
    )
    noisy_folder = (
        "/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy"
    )

    print("Loading data info ...")
    # Match multiple image file extensions
    file_patterns = [
        os.path.join(noisy_folder, f"*.{ext}") for ext in ["png", "jpeg", "jpg", "bmp"]
    ]
    noisy_files = []
    for pattern in file_patterns:
        noisy_files.extend(glob.glob(pattern))
    noisy_files.sort()

    psnr_test = 0
    num_images = len(noisy_files)

    for f in noisy_files:
        filename = os.path.basename(f)
        clean_file = os.path.join(clean_folder, filename)

        # Load noisy image
        Img_noisy = cv2.imread(f)
        if Img_noisy is None:
            print(f"Failed to load noisy image {f}")
            continue

        # Convert color image to grayscale if necessary
        if Img_noisy.ndim == 3 and Img_noisy.shape[2] == 3:
            Img_noisy = cv2.cvtColor(Img_noisy, cv2.COLOR_BGR2GRAY)

        Img_noisy = normalize(np.float32(Img_noisy))
        Img_noisy = np.expand_dims(Img_noisy, 0)  # Add channel dimension
        Img_noisy = np.expand_dims(Img_noisy, 1)  # Add batch dimension
        INoisy = torch.Tensor(Img_noisy)

        # Load clean image
        Img_clean = cv2.imread(clean_file)
        if Img_clean is None:
            print(f"Failed to load clean image {clean_file}")
            continue

        # Convert color image to grayscale if necessary
        if Img_clean.ndim == 3 and Img_clean.shape[2] == 3:
            Img_clean = cv2.cvtColor(Img_clean, cv2.COLOR_BGR2GRAY)

        Img_clean = normalize(np.float32(Img_clean))
        Img_clean = np.expand_dims(Img_clean, 0)  # Add channel dimension
        Img_clean = np.expand_dims(Img_clean, 1)  # Add batch dimension
        ISource = torch.Tensor(Img_clean)

        ISource, INoisy = Variable(ISource.cuda()), Variable(INoisy.cuda())

        with torch.no_grad():
            Out = torch.clamp(INoisy - model(INoisy), 0.0, 1.0)
        psnr = batch_PSNR(Out, ISource, 1.0)
        psnr_test += psnr
        print(f"{f} PSNR: {psnr:.4f}")

    psnr_test /= num_images
    print(f"\nPSNR on test data: {psnr_test:.4f}")


In [ ]:
test_dncnn(
    test_data="/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy",  # The directory containing your test images
    num_of_layers=17,  # The number of layers in your DnCNN model
    logdir="./logs",  # The directory where your model weights are saved
)

Loading model ...
Loading data info ...
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/10.bmp PSNR: 3.9288
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/11.bmp PSNR: 3.9405
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/15.bmp PSNR: 3.9342
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/2.bmp PSNR: 3.8386
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/22.bmp PSNR: 3.9601
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/25.bmp PSNR: 4.1138
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/26.bmp PSNR: 4.1535
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/29.bmp PSNR: 4.2138
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/31.bmp PSNR: 4.0201
/home/kareem/hacking/research/AI_Love/denoising/noise2noise_data/test/noisy/35.bmp PSNR: 4.0466
/